In [171]:
import numpy as np
import pandas as pd
import random
import ast
from tqdm import tqdm
from IPython import display as ipd

# visualization
import matplotlib.pyplot as plt
from tabulate import tabulate
import seaborn as sns
#system files
import os
import json
import re
import glob

import zipfile
import shutil
import gc
from joblib import Parallel, delayed

from datasets import load_metric
#normalization
from pandarallel import pandarallel
from bnunicodenormalizer import Normalizer 
pandarallel.initialize(progress_bar=True,nb_workers=8)
tqdm.pandas()
bnorm=Normalizer()

# Set environment variables
import warnings
warnings.filterwarnings('ignore')

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [172]:
df=pd.read_csv('final_with_punctuation_needs_cleaning.csv')

In [173]:
df = df.drop(list(df.columns)[0],axis=1)

In [174]:
df = df.rename(columns={'punctuation':'sentence'})

In [175]:
df

,path,sentence
0,common_voice_bn_31584094.mp3,"দ্বিতীয়ত, এটি বহুল প্রচলিত একটি রিজার্ভ কারেন্সী।"
1,common_voice_bn_31617509.mp3,জ্ঞান চর্চার সুবিধার্থে পাঠাগার আইসিটি চর্চার জন্য একটি কম্পিউটার ল্যাব রয়েছে।
2,common_voice_bn_31529519.mp3,এর উত্তর দিকে রয়েছে বরণীয় দ্বীপ। দক্ষিণে রয়েছে জাভা দ্বীপ।
3,common_voice_bn_31694167.mp3,পরের বছর তিনজন কৃষ্ণাঙ্গ ব্যক্তি আইনসভায় নির্বাচিত হন।
4,common_voice_bn_30999394.mp3,কলেজটির নিজস্ব ক্যাম্পাস ভবন নেই।
...,...,...
7742,common_voice_bn_31574515.mp3,পিতার দিক থেকে তিনি পোলীয় ও রুশ এবং মায়ের দিক থেকে তিনি হাঙ্গেরি ও পোলীয় বংশোদ্ভূত।
7743,common_voice_bn_31511061.mp3,এই শহরের নাম বৌদ্ধ ধর্মগুরু পয়গম্বরের নাম থেকে হয়েছে।
7744,common_voice_bn_31561913.mp3,দুই পক্ষেই রাক্ষস নায়কদের বিশেষ ভূমিকা ছিল।
7745,common_voice_bn_31524603.mp3,তিনি আইনজীবী হিসেবে কর্মজীবন শুরু করেন।


In [176]:
def cleaning_punctuation(sen):
    if len(sen) != 0:
        sen=sen.strip()
        all_words=[]
        for word in sen.split()[:-1]:
            if '।' in word:
                all_words.append(word[:-1]+',')
            else:
                all_words.append(word)
        final_word = sen.split()[-1]
        if final_word[-1] != '।':
            if final_word[-1] == '?':
                all_words.append(final_word)
            if final_word[-1] == ',':
                all_words.append(final_word[:-1]+'।')
            if final_word[-1] not in [',','?']:
                all_words.append(final_word+'।')
        else:
            all_words.append(final_word)
    else:
        return sen
    return " ".join(all_words)

In [177]:
df['sentence'] = df['sentence'].str.replace('.', '')

In [178]:
df['sentence'] = df['sentence'].apply(lambda x:cleaning_punctuation(x))

In [179]:
pd.set_option('display.max_colwidth', None)

In [180]:
df[df['sentence'].str.find(',') != -1]

,path,sentence
0,common_voice_bn_31584094.mp3,"দ্বিতীয়ত, এটি বহুল প্রচলিত একটি রিজার্ভ কারেন্সী।"
2,common_voice_bn_31529519.mp3,"এর উত্তর দিকে রয়েছে বরণীয় দ্বীপ, দক্ষিণে রয়েছে জাভা দ্বীপ।"
5,common_voice_bn_31609225.mp3,"ওরা যেসব শরিক সাব্যস্ত করেছে, শেষ থেকে তিনি পবিত্র, বহু অর্ধে।"
41,common_voice_bn_31557568.mp3,"আমেরিকা, অস্ট্রেলিয়া ও আফ্রিকায় এভাবেই পাখিটির আগমণ ঘটেছে।"
42,common_voice_bn_31719038.mp3,"সম্প্রদায়গুলির নিজস্ব কিছু আচার আচরণ থাকে, যা একটি থেকে অপরটি ভিন্ন।"
...,...,...
7708,common_voice_bn_31735989.mp3,"ইসলামে রোমানিয়ার বিশ্বাসীদের অধিকাংশই সুন্নি, যারা হানাফী মাঝ হাব মেনে চলে।"
7710,common_voice_bn_31731258.mp3,"জঙ্গলের মধ্যে সেই হোটেল লীগে তিনি জানতে পারেন, কিছুদিন পূর্বে দুই পর্যটক এখান থেকে নিকো হয়ে গেছে।"
7721,common_voice_bn_31651490.mp3,"তিনি আইনজীবী, রাজনীতি এবং বাংলাদেশের রাষ্ট্রপতি ছিলেন।"
7738,common_voice_bn_31661205.mp3,"ঐ বছরই তিনি পার লিন নগরে চলে আসেন, আরো ভালোভাবে সংস্কৃত শিক্ষালাভের জন্য।"


In [181]:
df[df['sentence'].str.find('?') != -1]

,path,sentence
781,common_voice_bn_30994440.mp3,"তখন সে এই ভগ্নম্বরশ বা দশমিক সংখ্যা কে কল্পনা করতে পারে অতিক্রান্ত, দুরুত্ব বা অন্যান্য উদাহরণ দিয়ে?"
901,common_voice_bn_31563559.mp3,"প্রশ্ন হচ্ছে, সমালোচনামূলক ভূগোল এখনও রাজনৈতিক পরিবর্তনের প্রতি প্রতিশ্রুতিবদ্ধ কিনা?"
1254,common_voice_bn_31514319.mp3,"প্রতিপক্ষ দল যদি খেলাকে সংক্ষিপ্ত করতে চায়, তা হল আমি কিভাবে দীর্ঘায়িত করবো?"
1510,common_voice_bn_31533812.mp3,"তাই তাকে প্রথমে ভালো ভাবে বুঝিয়ে বলেন, তাদের পরিকল্পনা কি ছিল?"
1987,common_voice_bn_31661818.mp3,"এগুলি পবিত্র ভবনের একটি বিশেষ রূপ, কোনটি পৃথক প্রতিষ্ঠান বা কোনটি বৃহৎ মসজিদের সাথে সংযুক্ত?"
2203,common_voice_bn_31637947.mp3,তিনি তার অপরাধ কোন কর্মকাণ্ডের জন্য ভারতের বিভিন্ন রাজ্য দেশের দ্বারা করতে ছিলেন?
2298,common_voice_bn_31677555.mp3,কোন ট্রেন পরিষদ নেই?
2430,common_voice_bn_31637530.mp3,"প্রথাগত লোক সংস্কৃতি সবসময় মূলত এক একটি পরিবেশনা, হকস এটি কোন একক অঙ্গভঙ্গি বা জুটির লিখে তৈরি? তিনি তিন।"
3023,common_voice_bn_31622100.mp3,"উবাই জিজ্ঞেস করলেন, তখন আপনি কি করেছেন?"
3196,common_voice_bn_31548806.mp3,কার কি কি ঔষধ লাগবে?


In [182]:
question_id=df.index[df['sentence'].str.find('?') == True]

In [183]:
comma_id=df.index[df['sentence'].str[-1] == ',']

In [186]:
len(question_id)

0

In [185]:
comma_id

Int64Index([], dtype='int64')

In [187]:
df.to_csv('final_attempt.csv',index=False)

In [116]:
nan_id=df.loc[pd.isna(df["sentence"]), :].index

In [73]:
nan_id

Int64Index([], dtype='int64')

In [135]:
df.iloc[comma_id]

,path,sentence
5,common_voice_bn_31609225.mp3,"ওরা যেসব শরিক সাব্যস্ত করেছে, শেষ থেকে তিনি পবিত্র,"
270,common_voice_bn_31618978.mp3,"ঢাকা থেকে সাভার যাওয়ার পথেই পড়ে বাংলাদেশের সর্বজন থিম পার্ক,"
462,common_voice_bn_31627764.mp3,"জীবনে চলার পথে তারা মুখ থুবড়ে পড়ে থাকে,"
601,common_voice_bn_31576711.mp3,"যেমনএকদুই তিন,"
814,common_voice_bn_31660319.mp3,"মুসলিম জনসংখ্যার সর্বাধিক ঘনত্ব জার্মান ভাষা,"
909,common_voice_bn_31675935.mp3,"প্রতিরক্ষা মন্ত্রী,"
979,common_voice_bn_31625466.mp3,"এর পরের মাসেই তিনি টেস্ট ক্রিকেটে অভিষিক্ত হন,"
1130,common_voice_bn_31524747.mp3,"দলে তিনি মূলতঃ বোলারের দায়িত্ব পালন করেছেন,"
1256,common_voice_bn_31716290.mp3,"পদের ইংরাজি পত্রিকা হিন্দু স্থান স্ট্যান্ডার্ড এ আকতে থাকেন,"
1398,common_voice_bn_31599210.mp3,"মানে একটা অ্যালকোহল,"


In [89]:
deleted = df[df.index.isin(nan_id)]['punctuation'].tolist()

In [90]:
df=df.drop(nan_id)
df=df.reset_index(drop=True)

In [91]:
df

,path,punctuation
0,common_voice_bn_31584094.mp3,"দ্বিতীয়ত, এটি বহুল প্রচলিত একটি রিজার্ভ কারেন্সী"
1,common_voice_bn_31617509.mp3,জ্ঞান চর্চার সুবিধার্থে পাঠাগার আইসিটি চর্চার জন্য একটি কম্পিউটার ল্যাব রয়েছে
2,common_voice_bn_31529519.mp3,এর উত্তর দিকে রয়েছে বরণীয় দ্বীপদক্ষিণে রয়েছে জাভা দ্বীপ
3,common_voice_bn_31694167.mp3,পরের বছর তিনজন কৃষ্ণাঙ্গ ব্যক্তি আইনসভায় নির্বাচিত হন
4,common_voice_bn_30999394.mp3,কলেজটির নিজস্ব ক্যাম্পাস ভবন নেই
...,...,...
7742,common_voice_bn_31574515.mp3,পিতার দিক থেকে তিনি পোলীয় ও রুশ এবং মায়ের দিক থেকে তিনি হাঙ্গেরি ও পোলীয় বংশোদ্ভূত
7743,common_voice_bn_31511061.mp3,এই শহরের নাম বৌদ্ধ ধর্মগুরু পয়গম্বরের নাম থেকে হয়েছে
7744,common_voice_bn_31561913.mp3,দুই পক্ষেই রাক্ষস নায়কদের বিশেষ ভূমিকা ছিল
7745,common_voice_bn_31524603.mp3,তিনি আইনজীবী হিসেবে কর্মজীবন শুরু করেন


In [91]:
df['sentence'][1][-1] == ','

False

In [94]:
for i in range(len(df)):
    if df['sentence'][i][-1] == ',':
        #df['sentence'][i] = df['sentence'][i][:-1]
        print(df['sentence'][i])

IndexError: string index out of range

In [92]:
df

,path,punctuation
0,common_voice_bn_31584094.mp3,"দ্বিতীয়ত, এটি বহুল প্রচলিত একটি রিজার্ভ কারেন্সী"
1,common_voice_bn_31617509.mp3,জ্ঞান চর্চার সুবিধার্থে পাঠাগার আইসিটি চর্চার জন্য একটি কম্পিউটার ল্যাব রয়েছে
2,common_voice_bn_31529519.mp3,এর উত্তর দিকে রয়েছে বরণীয় দ্বীপদক্ষিণে রয়েছে জাভা দ্বীপ
3,common_voice_bn_31694167.mp3,পরের বছর তিনজন কৃষ্ণাঙ্গ ব্যক্তি আইনসভায় নির্বাচিত হন
4,common_voice_bn_30999394.mp3,কলেজটির নিজস্ব ক্যাম্পাস ভবন নেই
...,...,...
7742,common_voice_bn_31574515.mp3,পিতার দিক থেকে তিনি পোলীয় ও রুশ এবং মায়ের দিক থেকে তিনি হাঙ্গেরি ও পোলীয় বংশোদ্ভূত
7743,common_voice_bn_31511061.mp3,এই শহরের নাম বৌদ্ধ ধর্মগুরু পয়গম্বরের নাম থেকে হয়েছে
7744,common_voice_bn_31561913.mp3,দুই পক্ষেই রাক্ষস নায়কদের বিশেষ ভূমিকা ছিল
7745,common_voice_bn_31524603.mp3,তিনি আইনজীবী হিসেবে কর্মজীবন শুরু করেন


In [67]:
df['punctuation'][0][-1] not in [',','?']

True

In [93]:
for i in range(len(df)):
    if df['punctuation'][i][-1] in [',','?']:
        print(df['punctuation'][i])

In [68]:
for i in range(len(df)):
    if df['punctuation'][i][-1] not in [',','?']:
        df['punctuation'][i] = df['punctuation'][i] + '।'

IndexError: string index out of range

In [94]:
df

,path,punctuation
0,common_voice_bn_31584094.mp3,"দ্বিতীয়ত, এটি বহুল প্রচলিত একটি রিজার্ভ কারেন্সী"
1,common_voice_bn_31617509.mp3,জ্ঞান চর্চার সুবিধার্থে পাঠাগার আইসিটি চর্চার জন্য একটি কম্পিউটার ল্যাব রয়েছে
2,common_voice_bn_31529519.mp3,এর উত্তর দিকে রয়েছে বরণীয় দ্বীপদক্ষিণে রয়েছে জাভা দ্বীপ
3,common_voice_bn_31694167.mp3,পরের বছর তিনজন কৃষ্ণাঙ্গ ব্যক্তি আইনসভায় নির্বাচিত হন
4,common_voice_bn_30999394.mp3,কলেজটির নিজস্ব ক্যাম্পাস ভবন নেই
...,...,...
7742,common_voice_bn_31574515.mp3,পিতার দিক থেকে তিনি পোলীয় ও রুশ এবং মায়ের দিক থেকে তিনি হাঙ্গেরি ও পোলীয় বংশোদ্ভূত
7743,common_voice_bn_31511061.mp3,এই শহরের নাম বৌদ্ধ ধর্মগুরু পয়গম্বরের নাম থেকে হয়েছে
7744,common_voice_bn_31561913.mp3,দুই পক্ষেই রাক্ষস নায়কদের বিশেষ ভূমিকা ছিল
7745,common_voice_bn_31524603.mp3,তিনি আইনজীবী হিসেবে কর্মজীবন শুরু করেন


In [26]:
for i in range(len(deleted)):
    df = df.append({'path': deleted[i],'punctuation':'',},ignore_index=True)

In [29]:
df=df.rename(columns={'punctuation':'sentence'})

In [30]:
df.to_csv('4.csv',index=False)

In [12]:
df=pd.read_csv('lessgo1.csv')

In [13]:
a=df['sentence'][0]

In [14]:
" ".join(a.split())

'দ্বিতীয়ত, এটি বহুল প্রচলিত একটি রিজার্ভ কারেন্সী।'

In [16]:
def space_remover(x):
    return " ".join(x.split())

In [17]:
space_remover(a)

'দ্বিতীয়ত, এটি বহুল প্রচলিত একটি রিজার্ভ কারেন্সী।'

In [21]:
type(a)

str

In [24]:
df['sentence']=df['sentence'].apply(lambda x: space_remover(str(x)))

In [30]:
df.to_csv('lessgo2.csv',index=False)